# Redes neuronales (ejercicio)

**Autor**: Sergio Rodríguez Calvo

Vamos a comenzar con las importaciones:

In [1]:
import tensorflow as tf
from sklearn.neural_network import MLPClassifier

### **Importante: comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría.

## Parte 1: aplicación de redes neuronales a clasificación (análisis de sentimientos)

Se pide aplicar un modelo de redes neuronales al problema de decidir si una crítica de cine es positiva o negativa. Para ello volvemos a usar los datos de IMDB (Internet Movie Database) que vimos en el módulo 2 (modelo probabilístico).

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPClassifier`
* Keras con Tensorflow: usar `Sequential` y capas tipo `Dense` con la arquitectura adecuda.


Aunque ya hemos visto que los datos están disponibles en http://ai.stanford.edu/~amaas/data/sentiment/ , en este caso pedimos cargar los datos usando la utilidad `imdb`de Keras. Se puede consultar en el manual de Keras: https://keras.io/datasets/ Cargarlos con `imdb.load_data` y usar los datos cargados como punto de partida a este ejercicio (tanto para su aplicar scikit learn como para aplicar keras). Prestar atención al formato en el que se cargar, que no es el mismo que hamos visto hasta ahora.  

Los textos han de ser vectorizados para que se puedan ser procesados por una red. Para esto, tenemos varias alternativas, usar una de ellas:

* Vectorizando "manualmente", definiendo una función en python que lo haga.
* Vectorizadores de scikit learn (ya vistos)
* Herramientas de vectorización de keras: https://keras.io/preprocessing/text/

Mostrar algunas pruebas realizadas con distintas arquitecturas y/o hiperparámetros. No es necesario ser muy exhaustivo ni usar `GridSearchCV` en scikit learn ni el equivalente en keras. Tan solo mostrar alguna experimentación y ajuste manual.  

### Ejercicio

Vamos a cargar el _dataset_ de críticas de cine a partir de la librería Keras, para ello usaremos la función `load_data` que nos devuelve los conjuntos separados en entrenamiento y pruebas, así como, sus clasificaciones respectivamente.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data()

Vamos a ver la forma de los datos para ambos conjuntos:

In [3]:
x_train.shape, x_test.shape

((25000,), (25000,))

Cada conjunto cuenta con 25.000 ejemplos, en este caso, con una sola columna correspondiente a el texto con la crítica. Además, el número de columnas es variable, por eso no sale información, por lo que vamos a necesitar de un preprocesado para tener el mismo número de columnas en todas las instancias. Ahora, vamos a echar un vistazo a una de las instancias:

In [4]:
print(x_train[100])

[1, 13, 244, 6, 87, 337, 7, 628, 2219, 5, 28, 285, 15, 240, 93, 23, 288, 549, 18, 1455, 673, 4, 241, 534, 3635, 8448, 20, 38, 54, 13, 258, 46, 44, 14, 13, 1241, 7258, 12, 5, 5, 51, 9, 14, 45, 6, 762, 7, 17802, 1309, 328, 5, 428, 2473, 15, 26, 1292, 5, 3939, 6728, 5, 1960, 279, 13, 92, 124, 803, 52, 21, 279, 14, 9, 43, 6, 762, 7, 595, 15, 16, 28911, 23, 4, 1071, 467, 4, 403, 7, 628, 2219, 8, 97, 6, 171, 3596, 99, 387, 72, 97, 12, 788, 15, 13, 161, 459, 44, 4, 3939, 1101, 173, 21, 69, 8, 401, 22239, 4, 481, 88, 61, 4731, 238, 28, 32, 11, 32, 14, 9, 6, 545, 1332, 766, 5, 203, 73, 28, 43, 77, 317, 11, 4, 22228, 953, 270, 17, 6, 3616, 13, 545, 386, 25, 92, 1142, 129, 278, 23, 14, 241, 46, 7, 158]


Como se comenta en la descripción, las críticas o revisiones han sido preprocesadas por lo que cada crítica está codificada como una lista de palabras indexadas (enteros). Esto es, cada palabra está codificada por la frecuencia sobre el dataset al completo (si el entero para esa palabra es 3, eso significa que es la tercera palabra más frecuente).

Esto permite un filtrado rapido de tipo "elimina las 20 palabras más comunes". Al cargar los datos, existe un parámetro `skip_top` que permite realizar este proceso de manera automática mientras se realiza la carga. En este experimento, por tanto, vamos a evitar filtrado alguno ya que se ha usado el parámetro por defecto a 0, y vamos a utilizar los datos con la vectorización por defecto, la que acabamos de describir.

Además, los datos vienen clasificados como positivos (0) o negativos (1).

Vamos a obtener el mensaje original para aplicar una vectorización a los mensajes como se ha visto en ejercicios previos. Para ello, vamos a ser uso del índice de palabras del dataset, vamos a crear un diccionario para poder realizar este proceso a partir de los indices de las palabras y, finalmente, vamos a crear una función que decodifique cada crítica.

In [5]:
word_index = tf.keras.datasets.imdb.get_word_index()
rev_word_index = {idx:w for w,idx in word_index.items()}

def decode_sentence(s):
    # index 0 to 2 are reserved for things like padding, unknown word, etc.
    decoded_sent = [rev_word_index.get(idx-3, '') for idx in s]
    return ' '.join(decoded_sent)

print(decode_sentence(x_train[100]))

 i am a great fan of david lynch and have everything that he's made on dvd except for hotel room the 2 hour twin peaks movie so when i found out about this i immediately grabbed it and and what is this it's a bunch of crudely drawn black and white cartoons that are loud and foul mouthed and unfunny maybe i don't know what's good but maybe this is just a bunch of crap that was foisted on the public under the name of david lynch to make a few bucks too let me make it clear that i didn't care about the foul language part but had to keep adjusting the sound because my neighbors might have all in all this is a highly disappointing release and may well have just been left in the deluxe box set as a curiosity i highly recommend you don't spend your money on this 2 out of 10


Además, hemos mostrado una crítica del conjunto de entrenamiento, la misma que mostramos anteriormente como vector de números.

A continuación, vamos a decodificar todos los mensajes y a obtener un conjunto de entrenamiento decodificado, así como un conjunto de pruebas decodificado. Para ello, vamos a usar la función `decode_sentence` definida previamente.

In [6]:
x_train_decode = []
for sentence in x_train:
    decode_sent = decode_sentence(sentence)
    x_train_decode.append(decode_sent)

print(x_train_decode[100])

 i am a great fan of david lynch and have everything that he's made on dvd except for hotel room the 2 hour twin peaks movie so when i found out about this i immediately grabbed it and and what is this it's a bunch of crudely drawn black and white cartoons that are loud and foul mouthed and unfunny maybe i don't know what's good but maybe this is just a bunch of crap that was foisted on the public under the name of david lynch to make a few bucks too let me make it clear that i didn't care about the foul language part but had to keep adjusting the sound because my neighbors might have all in all this is a highly disappointing release and may well have just been left in the deluxe box set as a curiosity i highly recommend you don't spend your money on this 2 out of 10


In [7]:
x_test_decode = []
for sentence in x_test:
    decode_sent = decode_sentence(sentence)
    x_test_decode.append(decode_sent)

print(x_test_decode[100])

 a quick glance at the premise of this film would seem to indicate just another dumb '80's inbred backwood slash fest the type where sex equals death and the actors are all annoying stereotypes you actually want to die however delivers considerably more br br rather than focus on bare flesh and gore though there is a little of each no sex however the flick focuses on delivering impending dread mounting tension amidst a lovely scenic backdrop these feelings are further heightened by a cast of realistically likable characters and antagonists that are more amoral than cardboard definitions of evil oh yeah george kennedy is here too and when is that not a good thing br br if you liked wrong turn then watch this to see where much of its' methodology came from


El siguiente paso es vectorizar los conjuntos de entrenamiento y pruebas, para ello, vamos a importar y utilizar `CountVectorizer` al igual que hicimos en ejercicios anteriores.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(x_train_decode)

Una vez entrenado el vectorizador, procedemos a obtener los conjuntos de entrenamiento y pruebas vectorizado. También, vamos a indicar las dimensiones de la matriz dispersa obtenida.

In [9]:
X_train = vect.transform(x_train_decode)
X_test = vect.transform(x_test_decode)

print("X_train:\n{}".format(repr(X_train)))

X_train:
<25000x74702 sparse matrix of type '<class 'numpy.int64'>'
	with 3445804 stored elements in Compressed Sparse Row format>


La matriz de entrenamiento tiene tantas filas como críticas o revisiones habia en el conjunto original y cuenta con 74.702 componentes, todos del mismo tamaño, que no es más que un vector de ceros y unos indicando cual de las palabras del corpus forman parte de dicha instancia.

Entonces, ya podemos comenzar el experimento utilizando `MLPClassifier` de scikit-learn. Para ello, vamos a crear el modelo y a entrenarlo directamente. Vamos a usar los parámetros por defecto salvo los que se modifican en la instancia del modelo, que son:
* `solver`: vamos a mantener el mismo algoritmo de optimización de los pesos que en el ejercicio visto en la asignatura, ya que estabamos en un problema de similares características, es decir, clasificación binaria.
* `random_state`: ya se ha visto en ejercicios anteriores, es una forma de fijar la inizialización del modelo. De esta forma, siempre obtendremos el mismo resultado. Es algo que usaremos a lo largo de la práctica allí donde se pueda.

In [10]:
mlp = MLPClassifier(solver='lbfgs', random_state=0).fit(X_train, y_train)

print("Rendimiento en entenamiento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp.score(X_test, y_test)))

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Rendimiento en entenamiento: 0.99
Rendimiento en el conjunto de prueba: 0.86


Ahora, vamos a modificar el parámetro `alpha` para ver si obtenemos un resultado diferente del anterior.

In [11]:
mlp_alpha_1 = MLPClassifier(solver='lbfgs', alpha=1, random_state=0).fit(X_train, y_train)

print("Rendimiento en entenamiento: {:.2f}".format(mlp_alpha_1.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp_alpha_1.score(X_test, y_test)))

/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Rendimiento en entenamiento: 0.99
Rendimiento en el conjunto de prueba: 0.85


keras

In [12]:
import tensorflow.keras as keras
model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="sigmoid"))

2021-12-28 00:39:05.669098: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

In [14]:
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [15]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30


/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 300), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


625/625 [==============================] - 7s 10ms/step - loss: 0.6684 - accuracy: 0.6874 - val_loss: 0.5671 - val_accuracy: 0.7332
Epoch 2/30
625/625 [==============================] - 7s 12ms/step - loss: 0.4946 - accuracy: 0.7879 - val_loss: 0.4707 - val_accuracy: 0.7880
Epoch 3/30
625/625 [==============================] - 9s 14ms/step - loss: 0.4260 - accuracy: 0.8216 - val_loss: 0.5683 - val_accuracy: 0.7480
Epoch 4/30
625/625 [==============================] - 9s 15ms/step - loss: 0.3882 - accuracy: 0.8424 - val_loss: 0.3766 - val_accuracy: 0.8396
Epoch 5/30
625/625 [==============================] - 9s 15ms/step - loss: 0.3472 - accuracy: 0.8586 - val_loss: 0.3354 - val_accuracy: 0.8636
Epoch 6/30
625/625 [==============================] - 11s 18ms/step - loss: 0.3276 - accuracy: 0.8691 - val_loss: 0.3586 - val_accuracy: 0.8530
Epoch 7/30
625/625 [==============================] - 11s 18ms/step - loss: 0.3134 - accuracy: 0.8752 - val_loss: 0.3236 - val_accuracy: 0.8692
Epoch 8/

In [16]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 9s 11ms/step - loss: 0.3997 - accuracy: 0.8723


[0.3997378647327423, 0.8722800016403198]

## Parte 2: aplicación de redes neuronales a regresión (predicción del precio de la vivienda)

Se pide aplicar un modelo de redes neuronales al problema de predecir precios de vivienda usando el conjunto de datos  `Boston house prices`. 

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPRegressor`
* Keras con Tensorflow: usar nuevamente `Sequential` y capas tipo `Dense` con la arquitectura adecuada.

El conjunto de datos se puede cargar usando tanto scikit learn (`sklearn.datasets.load_boston`) como keras (`keras.datasets.boston_housing`). 

Como en la parte 1, se pide mostrar algunas pruebas de los resultados obtenidos usando distintas arquitecturas y/o hiperparámetros. 


### Ejercicio